In [25]:
import pandas as pd
import utilssphering
import utils
import os
import random
import numpy as np

random.seed(9000)

In [12]:
# Set some variables + read the metadata

n_samples = 1000
n_replicates = 4  #number of sample replicates within each plate 
metadata_common = 'Metadata_moa'
metadata_perturbation = 'Metadata_broad_sample'
group_by_feature = 'Metadata_broad_sample'

experiment_df = (
    pd.read_csv('output/experiment-metadata.tsv', sep='\t')
#    .query('Batch==@batch')
)


In [13]:
### Trying to find cell counts

for a_vendor in experiment_df["Vendor"].unique():
    vendor_data = experiment_df.loc[experiment_df["Vendor"] == a_vendor]
#    print([a_vendor, len(vendor_data)])
    for a_batch in vendor_data["Batch"].unique():
        batch_data = vendor_data.loc[vendor_data["Batch"] == a_batch]
#        print([a_batch, len(batch_data)])
        for a_plate in batch_data["Assay_Plate_Barcode"].unique():
            plate_data = batch_data.loc[batch_data["Assay_Plate_Barcode"] == a_plate]
#            print([a_plate, len(plate_data)])
            plate_df = utils.load_data(a_batch, a_plate, "normalized_feature_select_negcon_batch.csv.gz")
            print(plate_df.columns)

Index(['Metadata_broad_sample', 'Metadata_solvent', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_InChIKey', 'Metadata_pert_iname',
       'Metadata_pubchem_cid', 'Metadata_moa', 'Metadata_pert_type',
       'Metadata_control_type',
       ...
       'Nuclei_Texture_Correlation_RNA_20_02_256',
       'Nuclei_Texture_DifferenceEntropy_RNA_5_00_256',
       'Nuclei_Texture_DifferenceVariance_Golgi_20_01_256',
       'Nuclei_Texture_DifferenceVariance_RNA_20_03_256',
       'Nuclei_Texture_Entropy_Mito_20_01_256',
       'Nuclei_Texture_InfoMeas1_Actin_20_01_256',
       'Nuclei_Texture_InfoMeas1_Actin_5_00_256',
       'Nuclei_Texture_InfoMeas1_DNA_10_03_256',
       'Nuclei_Texture_InfoMeas1_Golgi_20_01_256',
       'Nuclei_Texture_SumVariance_Mito_20_03_256'],
      dtype='object', length=504)
Index(['Metadata_broad_sample', 'Metadata_solvent', 'Metadata_Plate',
       'Metadata_Well', 'Metadata_InChIKey', 'Metadata_pert_iname',
       'Metadata_pubchem_cid', 'Metadata_moa', 'Met

In [20]:
dire = "../jump-scope/profiles/1siteSubSample_Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA"

for i in os.listdir(dire):
    if i.endswith(".csv.gz"):
        print(i)
        t = pd.read_csv(os.path.join("../jump-scope/profiles/1siteSubSample_Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA", i))
        # pd.read_csv("../jump-scope/profiles/1siteSubSample_Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA/Plate2_PCO_6ch_4site_10XPA_normalized_negcon.csv.gz")
        print([i for i in t.columns if "Metadata" in i])

# test = pd.read_csv("../jump-scope/profiles/1siteSubSample_Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA/Plate2_PCO_6ch_4site_10XPA.csv.gz")

# [i for i in plate_df.columns if "metadata_" in i.casefold()]

# test

Plate2_PCO_6ch_4site_10XPA_normalized_negcon.csv.gz
['Metadata_broad_sample', 'Metadata_solvent', 'Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_CellsSmall', 'Metadata_Count_Cytoplasm', 'Metadata_Count_CytoplasmSmall', 'Metadata_Count_Nuclei', 'Metadata_Count_NucleiSmall', 'Metadata_Object_Count', 'Metadata_InChIKey', 'Metadata_pert_iname', 'Metadata_pubchem_cid', 'Metadata_moa', 'Metadata_pert_type', 'Metadata_control_type', 'Metadata_smiles']
Plate2_PCO_6ch_4site_10XPA.csv.gz
['Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_CellsSmall', 'Metadata_Count_Cytoplasm', 'Metadata_Count_CytoplasmSmall', 'Metadata_Count_Nuclei', 'Metadata_Count_NucleiSmall', 'Metadata_Object_Count']
Plate2_PCO_6ch_4site_10XPA_normalized.csv.gz
['Metadata_broad_sample', 'Metadata_solvent', 'Metadata_Plate', 'Metadata_Well', 'Metadata_Site_Count', 'Metadata_Count_Cells', 'Metadata_Count_CellsSmall', 'Metadata_C

In [ ]:
# Original percentage calculations, without sphering

corr_replicating_df = pd.DataFrame()
corr_matching_df = pd.DataFrame()

for a_vendor in experiment_df["Vendor"].unique():
    vendor_data = experiment_df.loc[experiment_df["Vendor"] == a_vendor]
#    print([a_vendor, len(vendor_data)])
    for a_batch in vendor_data["Batch"].unique():
        batch_data = vendor_data.loc[vendor_data["Batch"] == a_batch]
#        print([a_batch, len(batch_data)])
        for a_plate in batch_data["Assay_Plate_Barcode"].unique():
            plate_data = batch_data.loc[batch_data["Assay_Plate_Barcode"] == a_plate]
#            print([a_plate, len(plate_data)])
            plate_df = utils.load_data(a_batch, a_plate, "normalized_feature_select_negcon_batch.csv.gz")
            plate_df = utils.remove_negcon_empty_wells(plate_df)
                
            replicating_corr = list(utils.corr_between_replicates(plate_df, group_by_feature))
            null_replicating = list(utils.corr_between_non_replicates(plate_df, n_samples=n_samples, n_replicates=n_replicates, metadata_compound_name = group_by_feature))

            prop_95_replicating, value_95_replicating = utils.percent_score(null_replicating, replicating_corr, how='right')
            
            corr_replicating_df = corr_replicating_df.append({'Vendor': a_vendor,
                                                              'Batch': a_batch,
                                                              'Assay_Plate_Barcode': a_plate,
                                                              'Replicating':replicating_corr,
                                                              'Null_Replicating':null_replicating,
                                                              'Percent_Replicating':'%.1f'%prop_95_replicating,
                                                              'Value_95':value_95_replicating}, ignore_index=True)
            
            matching_corr = list(utils.corr_between_perturbation_pairs(plate_df, 'Metadata_moa', 'Metadata_broad_sample'))
            null_matching = list(utils.corr_between_perturbation_non_pairs(plate_df, n_samples=n_samples, metadata_common=metadata_common, metadata_perturbation=metadata_perturbation))

            prop_95_matching, value_95_matching = utils.percent_score(null_matching, matching_corr, how='right')

            corr_matching_df = corr_matching_df.append({'Vendor': a_vendor,
                                                        'Batch': a_batch,
                                                        'Assay_Plate_Barcode': a_plate,
                                                        'Matching':matching_corr,
                                                        'Null_Matching':null_matching,
                                                        'Percent_Matching':'%.3f'%prop_95_matching,
                                                        'Value_95':value_95_matching}, ignore_index=True)

In [ ]:
# Reload any changes in packages (eg. utilssphering)
%load_ext autoreload
%autoreload 2

### Adding sphering
corr_replicating_df_sphere = pd.DataFrame()
corr_matching_df_sphere = pd.DataFrame()

for a_vendor in experiment_df["Vendor"].unique():
    vendor_data = experiment_df.loc[experiment_df["Vendor"] == a_vendor]
    for a_batch in vendor_data["Batch"].unique():
        batch_data = vendor_data.loc[vendor_data["Batch"] == a_batch]
        for a_plate in batch_data["Assay_Plate_Barcode"].unique():
            plate_data = batch_data.loc[batch_data["Assay_Plate_Barcode"] == a_plate]
            data_path = os.path.join("../jump-scope/profiles/", a_batch, a_plate, a_plate+"_normalized_feature_select_negcon_batch.csv.gz")
            load_data = pd.read_csv(data_path)

            # This works
            replicate_corr, null_corr, prop_95, value_95_replicating = utilssphering.calculate_percent_replicating_MOA("", "", data_df=load_data)

            matching_corr, matching_null_cor, prop_95_matching, value_95_matching = utilssphering.calculate_percent_matching_MOA("", "", data_df=load_data)
            
            corr_replicating_df_sphere = corr_replicating_df_sphere.append({'Vendor': a_vendor,
                                                              'Batch': a_batch,
                                                              'Assay_Plate_Barcode': a_plate,
                                                              'Replicating':replicate_corr,
                                                              'Null_Replicating':null_corr,
                                                              'Percent_Replicating':'%.1f'%prop_95,
                                                              'Value_95':value_95_replicating}, ignore_index=True)

            corr_matching_df_sphere = corr_matching_df_sphere.append({'Vendor': a_vendor,
                                                        'Batch': a_batch,
                                                        'Assay_Plate_Barcode': a_plate,
                                                        'Matching':matching_corr,
                                                        'Null_Matching':matching_null_cor,
                                                        'Percent_Matching':'%.3f'%prop_95_matching,
                                                        'Value_95':value_95_matching}, ignore_index=True)

In [33]:
experiment_df

,Vendor,Batch,Plate_Map_Name,Assay_Plate_Barcode,Modality,Binning,Magnification,Number_of_channels,Images_per_well,z_plane,BF_Zplanes,Anomaly
0,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_4site_10XPA,Confocal,1,10,6,4,1,NaN,none
1,MolDev,Scope1_MolDev_10X,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crest,Confocal,1,10,6,4,1,NaN,none
2,MolDev,Scope1_MolDev_10X_4siteZ,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_10XPA_Crestz,Confocal,1,10,6,4,3,NaN,none
3,MolDev,Scope1_MolDev_20X_4site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_4site_20XPA_Crestz,Confocal,1,20,6,4,3,NaN,none
4,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate2_PCO_6ch_9site_20XPA,Confocal,1,20,6,9,1,NaN,none
5,MolDev,Scope1_MolDev_20X_9site,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_9site_20XPA_Crest,Confocal,1,20,6,9,1,NaN,none
6,MolDev,Scope1_MolDev_20X_Adaptive,JUMP-MOA_compound_platemap,Plate3_PCO_6ch_Adaptive_20XPA,Confocal,1,20,6,3,1,NaN,none
7,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117060a10x,Widefield,1,10,4,1,1,NaN,none
8,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117061a10x,Widefield,1,10,4,1,1,NaN,none
9,Nikon,Scope1_Nikon_10X,JUMP-MOA_compound_platemap,BR00117062a10x,Widefield,1,10,4,1,1,NaN,none


In [30]:
# Merge 
# Set column names to take forward
columns = ['Vendor', 'Batch', 'Assay_Plate_Barcode']

# Rename columns to specify which results are sphered, or not
corr_replicating_df.rename(columns = {"Percent_Replicating": "Percent_Replicating_No_Sphere"}, inplace=True)
corr_matching_df.rename(columns = {"Percent_Matching": "Percent_Matching_No_Sphere"}, inplace=True)
corr_replicating_df_sphere.rename(columns = {"Percent_Replicating": "Percent_Replicating_Sphere"}, inplace=True)
corr_matching_df_sphere.rename(columns = {"Percent_Matching": "Percent_Matching_Sphere"}, inplace=True)

# Create a copy of the df to take forward 
_corr_replicating_df = corr_replicating_df[columns+['Percent_Replicating_No_Sphere']].copy()
_corr_matching_df = corr_matching_df[columns+['Percent_Matching_No_Sphere']].copy()
_corr_replicating_df_sphere = corr_replicating_df_sphere[columns+['Percent_Replicating_Sphere']].copy()
_corr_matching_df_sphere = corr_matching_df_sphere[columns+['Percent_Matching_Sphere']].copy()


# _corr_replicating_df is the df for everything to be merged to
rep_match_sphere_df = _corr_replicating_df.copy()

# Merge df on the aforementioned columns. Should be 100% row match
rep_match_sphere_df = rep_match_sphere_df.merge(
    _corr_matching_df, how="inner", on=columns).merge(
        _corr_replicating_df_sphere, how="inner", on=columns).merge(
            _corr_matching_df_sphere, how="inner", on=columns)

all_merge_df = pd.merge(
    rep_match_sphere_df, 
    experiment_df, 
    on = columns, 
    # left_on = ['Scope_vendor','Batch','Plate_Name'], 
    # right_on = ['Vendor','Batch','Assay_Plate_Barcode']
    )

# Save the df
rep_match_sphere_df.to_csv("replicating-matching-sphering.csv", index=False)
all_merge_df.to_csv("replicating-matching-sphering-with-modality-info.csv", index=False)

In [ ]:
# Subset a batch

rep_match_sphere_df.loc[rep_match_sphere_df["Batch"] == "Scope1_MolDev_10X"]


In [ ]:
# Dumbell plot the results to compare sphering

import pandas as pd
import matplotlib.pyplot as plt

rep_match_sphere_df = pd.read_csv("replicating-matching-sphering.csv")

fig, ax = plt.subplots(figsize=(5, 20), dpi=200)

df_len = rep_match_sphere_df.shape[0]

ax.set_xlim([10, 85])
ax.hlines(rep_match_sphere_df["Assay_Plate_Barcode"], 
        xmin=rep_match_sphere_df["Percent_Replicating_No_Sphere"].astype(float), 
        xmax=rep_match_sphere_df["Percent_Replicating_Sphere"].astype(float),
        color='grey', alpha=0.4)
ax.scatter(rep_match_sphere_df["Percent_Replicating_No_Sphere"].astype(float),
        rep_match_sphere_df["Assay_Plate_Barcode"],
        color='skyblue', alpha=1,
        label="No Sphere")
ax.scatter(rep_match_sphere_df["Percent_Replicating_Sphere"].astype(float),
        rep_match_sphere_df["Assay_Plate_Barcode"],
        color='green', alpha=0.4,
        label="Sphere")
for (_, row), y in zip(rep_match_sphere_df.iterrows(), range(0, df_len+1)):
        plt.annotate(f"+{row['Percent_Replicating_Sphere'] - row['Percent_Replicating_No_Sphere']:.1f}%", (max(row["Percent_Replicating_No_Sphere"], row["Percent_Replicating_Sphere"])+2, y - 0.2))
ax.legend(ncol=2, bbox_to_anchor=(1, 1.001), loc="lower right", frameon=False)
fig.set_facecolor("white")

fig.savefig("have_no_sphere.png", bbox_inches = 'tight')

In [22]:
### Creating dropout datasets + matrix

import pycytominer
import pandas as pd
import itertools

def do_feature_select(plate_df):
    """
    Find the column names that are CellProfiler features. Eg. column names
    that start with "Nuclei" or "Cytoplasm"
    """
    feature_select_features = pycytominer.cyto_utils.infer_cp_features(
        plate_df
    )
    # For all of the cellprofiler features, perform these operations on them
    return pycytominer.feature_select(
        profiles=plate_df,
        features=feature_select_features,
        operation=['variance_threshold','correlation_threshold',
        'drop_na_columns','blocklist']
    )

In [24]:

def all_combo_dropouts_in_a_category(category_list,outfile,subsample=None):
    """
    Create a database with dropped out features

    category_list: the categories to drop out (for example, Nuclei, Cytoplasm, Cells)
    outfile: save csv path
    """
    # For all of the categories, find all possible combinations with itertools.combinations
    to_try = [y for x in range(len(category_list)+1) for y in list(set(itertools.combinations(category_list,x)))]

    # If the output csv already exists, check what dropouts have already been tried
    # Don't try them again
    if os.path.exists(outfile):
        dropout_df = pd.read_csv(outfile)
        tried=list(dropout_df['Dropout'])
        print(f"Already try {tried},skipping")
        to_try = [x for x in to_try if str(x) not in tried]
    # Otherwise, dropout columns will be generated from scratch
    else:
        dropout_df = pd.DataFrame()
    
    # For the given categories, go through each combination (eg. ('Cells', 'Cytoplasm'), ('Nuclei', 'Cytoplasm') etc.)
    for eachdropout in to_try:
        # Get what columns are expected. I don't like this method
        normalized_master = pd.read_csv('../profiles-pilots/profiles/Stain5_CondC_Standard/BR00120274/BR00120274_normalized_negcon.csv.gz')
        col_list = normalized_master.columns
        if subsample:
             for each_item in subsample:
                metadata_list = [x for x in col_list if 'Metadata' in x] 
                col_list = metadata_list+[x for x in col_list if each_item in x]
        for each_item in eachdropout:
            # Create a list of columns that DOESN'T contain the categories being iterated
            # Thus this will iterate over all categories (eg. nuclei, cell, cytoplasm) and
            # drop all of the column names
            col_list = [x for x in col_list if each_item not in x]
        dropout_dict = {}
        temp_dropout_dict = {}
        # Code unique to this dataset. Iterate through plates
        for eachplate in range(70,78):
            try:
                # Read only the columns previously defined for each plate being iterated over
                # Again, don't like this hardcoding. Should be some sort of os.walk situation
                normalized = pandas.read_csv(
                    f'../profiles-pilots/profiles/Stain5_CondC_Standard/BR001202{eachplate}/BR001202{eachplate}_normalized_negcon.csv.gz',
                    usecols=col_list)
                # For the loaded dataset, calculate feature selection AFTER column dropout
                feature_select = do_feature_select(normalized)
                temp_dropout_dict[f'{eachplate}_featnum']=len(feature_select.columns)
                # Now, calculate the perc_rep for the dropout dataset 
                ### NOTE: in utilsphering, calculate_percent_replicating_MOA has been updated to return
                ### replicate_corr, null_corr, prop_95, value_95_replicating
                ### In this example, calculate_percent_replicating_MOA is **only returning prop_95**
                dropout_dict[f'BR001202{eachplate}']= calculate_percent_replicating_MOA('','',data_df=feature_select)
            except:
                dropout_dict[f'BR001202{eachplate}']=0
        # Convert the calculated perc_rep into np.array for each dropout 
        results_array = np.array(list(dropout_dict.values()))
        for k,v in temp_dropout_dict.items():
            dropout_dict[k]=v
        # For the dropout dataset created, calculate the mean perc_replicating for ALL of the plates
        # Why all of the plates? Are these replicates?
        # Means are calculated within each dropout group
        dropout_dict['Mean']=results_array.mean()
        dropout_dict['Median']=np.median(results_array)
        dropout_dict['Standard deviation']=np.std(results_array)
        dropout_dict['Dropout']=eachdropout
        dropout_dict['Remaining'] = [x for x in category_list if x not in eachdropout]
        dropout_dict['n_columns']=len(col_list)
        dropout_df = dropout_df.append(dropout_dict,ignore_index=True)
        print(eachdropout,dropout_dict['Mean'])
        dropout_df.to_csv(outfile,index=False)
    print(dropout_df['Mean'].describe())
    print(dropout_df['Median'].describe())


# all_combo_dropouts_in_a_category(['Nuclei','Cells','Cytoplasm'],
#                                  'checkpoint_csvs/compartment_dropout_combinations.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../profiles-pilots/profiles/Stain5_CondC_Standard/BR00120274/BR00120274_normalized_negcon.csv.gz'

In [54]:
def category_dropout(dataframe, dataframe_name, category_list, output):
    input_columns = dataframe.columns
    possible_combinations = [y for x in range(len(category_list)+1) for y in list(set(itertools.combinations(category_list,x)))]
    dropout_df = pd.DataFrame()

    # Iterate through the combination to dropout
    for dropout_group in possible_combinations: 
        col_list = input_columns
        # Within the droupout group, find the column to actually drop
        for each_item in dropout_group:
            # Only keep columns that don't contain the dropout
            col_list = [x for x in input_columns if each_item not in x]
            # print(col_list)
        print(dropout_group)
        dropped_dataframe = dataframe[col_list]
        # print(dataframe.shape, dropped_dataframe.shape)
        dropout_dict = {}
        # temp_dropout_dict = {}
        feature_select = do_feature_select(dropped_dataframe)
        dropout_dict[f"{dataframe_name}_num_features"] = len(feature_select.columns)
        _, _, prop_95, _ = utilssphering.calculate_percent_replicating_MOA("", "", feature_select)
        dropout_dict[f"{dataframe_name}_percent_replicating"] = prop_95

        results_array = np.array(list(dropout_dict.values()))
        # for k, v in temp_dropout_dict.items():
        #     dropout_dict[k] = v
        dropout_dict['Mean']=results_array.mean()
        dropout_dict['Median']=np.median(results_array)
        dropout_dict['Standard deviation']=np.std(results_array)
        dropout_dict['Dropout']=dropout_group
        dropout_dict['Remaining'] = [x for x in category_list if x not in dropout_group]
        dropout_dict['n_columns']=len(col_list)
        print(dropout_dict)

# TODO: Check that this is accurately dropping, integrate with os.walk, integrate with heatmap matrix


In [55]:
load_file = "../jump-scope/profiles/1siteSubSample_Scope1_MolDev_10X/Plate2_PCO_6ch_4site_10XPA/Plate2_PCO_6ch_4site_10XPA_normalized_negcon.csv.gz"

test_data = pd.read_csv(load_file)

category_dropout(test_data, "Plate2_PCO_6ch_4site_10XPA", ['Nuclei', 'Cells'], "t")


()
{'Plate2_PCO_6ch_4site_10XPA_num_features': 459, 'Plate2_PCO_6ch_4site_10XPA_percent_replicating': 45.55555555555556, 'Mean': 252.27777777777777, 'Median': 252.27777777777777, 'Standard deviation': 206.72222222222223, 'Dropout': (), 'Remaining': ['Nuclei', 'Cells'], 'n_columns': 4357}
('Nuclei',)
{'Plate2_PCO_6ch_4site_10XPA_num_features': 337, 'Plate2_PCO_6ch_4site_10XPA_percent_replicating': 42.22222222222222, 'Mean': 189.61111111111111, 'Median': 189.61111111111111, 'Standard deviation': 147.38888888888889, 'Dropout': ('Nuclei',), 'Remaining': ['Cells'], 'n_columns': 2909}
('Cells',)
{'Plate2_PCO_6ch_4site_10XPA_num_features': 392, 'Plate2_PCO_6ch_4site_10XPA_percent_replicating': 51.11111111111111, 'Mean': 221.55555555555554, 'Median': 221.55555555555554, 'Standard deviation': 170.44444444444446, 'Dropout': ('Cells',), 'Remaining': ['Nuclei'], 'n_columns': 2902}
('Nuclei', 'Cells')
{'Plate2_PCO_6ch_4site_10XPA_num_features': 392, 'Plate2_PCO_6ch_4site_10XPA_percent_replicating':